In [ ]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
import pickle as pkl
# if on google colab:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [ ]:
path_data = '/Bundesliga/'

In [ ]:
teilnehmer = dict()
teilnehmer[27362851] = 'Hacky'
teilnehmer[29318134] = 'Lars'
teilnehmer[29140453] = 'Dieter'
teilnehmer[26252098] = 'Henning'
teilnehmer[28320912] = 'Mirco'
teilnehmer[27383335] = 'Tobi89'
teilnehmer[29615386] = 'Johann-Lfbg'
teilnehmer[25768036] = 'Jörg'
teilnehmer[27922747] = 'OliverB'
teilnehmer[26242499] = 'onurb'
teilnehmer[26482402] = 'Schübbel'
teilnehmer[29140352] = 'Carina'
teilnehmer[26324257] = 'Oeschgie'
teilnehmer[26032575] = 'Seppi'
teilnehmer[30143645] = 'Tim'
teilnehmer[28082788] = 'MarcW'
teilnehmer[26333953] = 'Helga'
teilnehmer[26566383] = 'Jürgen'
teilnehmer[29227465] = 'Thomas'

In [ ]:
teams = {}
teams["FCA"] = 1
teams["FCB"] = 2
teams["ArmB"] = 3
teams["Herth"] = 4
teams["Boch"] = 5
teams["BVB"] = 6
teams["SGE"] = 7
teams["Frei"] = 8
teams["GrFü"] = 9
teams["Hoff"] = 10
teams["Köln"] = 11
teams["Leip"] = 12
teams["Lev"] = 13
teams["M05"] = 14
teams["BMG"] = 15 
teams["Stgt"] = 16
teams["Union"] = 17
teams["Wolf"] = 18

In [ ]:
def scrape_tipps(spieltag):
  tt = np.zeros((34,len(teilnehmer)+1,19))
  filename = path_data + 'tipps.pkl'
  try: # if file exists
    with open(filename, 'rb') as f:
        tt = pkl.load(f)
    for j in range(34):
      for i in range(9):
        if sum(sum(tt[j,1:,i+1:i+3])) == 0:
          print("ab Spieltag {} scrapen".format(j+1))
          s = j+1
          break
      if 's' in locals():
        break
  except:
    pass
  if 's' not in locals():
    s = 1
  for i in range(1,spieltag+1): 
      tipptable = np.zeros((len(teilnehmer)+1,19),dtype=float)
      URL = 'https://www.kicktipp.de/tippbuli-1718/tippuebersicht?&spieltagIndex=' + str(i)
      page = requests.get(URL)
      soup = BeautifulSoup(page.content, "html.parser")
      for j in range(9):
        spiel = soup.find(class_="ereignis nw ereignis" + str(j))
        team = BeautifulSoup(str(spiel), "html.parser")
        tea = team.find_all(class_="headerbox")
        tipptable[0][1+j*2] = teams[tea[0].text]
        tipptable[0][2+j*2] = teams[tea[1].text]
      # print(tipptable[0])
      tabelle = soup.find(class_="tippuebersicht ktable")
      tipper = BeautifulSoup(str(tabelle), "html.parser")
      tipper = tipper.find_all('tr')
      spiele = tipper[0]
      tipper = tipper[1:]

      for j in range(len(teilnehmer)):
        tipper1 = BeautifulSoup(str(tipper[j]), 'html.parser')
        name1 = tipper1.find(class_="mg_name").text
        tid = list(teilnehmer.keys())[list(teilnehmer.values()).index(str(name1))]
        tipptable[j+1][0] = tid
        tipper1 = tipper1.find_all('td')[3:12]
        import re
        for k in range(9):
          result = re.search('ereignis' + str(k) + '">(.*)<', str(tipper1[k]))
          try:
            result = str(result.group(1)).split('<')[0]
            tipptable[j+1][1+2*k] = result.split(':')[0]
            tipptable[j+1][2+2*k] = result.split(':')[1]
          except:
            tipptable[j+1][1+2*k] = np.nan
            tipptable[j+1][2+2*k] = np.nan
      tt[i-1] = tipptable
  with open(filename,'wb') as f:
    pkl.dump(tt, f)

In [ ]:
scrape_tipps(21)